# Ajuste fino (Fine-Tunne) de um Modelo de IA Generativa para Sumarização de Diálogos


Neste notebook, você irá fazer o fine-tune em um LLM existente da Hugging Face para melhorar a sumarização de diálogos. Você utilizará o modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) , que oferece um modelo ajustado de alta qualidade e capaz de sumarizar texto diretamente. Para aprimorar as inferências, você explorará uma abordagem completa de fine-tuning e avaliará os resultados com métricas ROUGE. Em seguida, você realizará o Parameter Efficient Fine-Tuning (PEFT), avaliará o modelo resultante e verá que os benefícios do PEFT superam ligeiramente as métricas de desempenho mais baixas.


# Table of Contents

- [ 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM](#1)
  - [ 1.1 - Set up Kernel and Required Dependencies](#1.1)
  - [ 1.2 - Load Dataset and LLM](#1.2)
  - [ 1.3 - Test the Model with Zero Shot Inferencing](#1.3)
- [ 2 - Perform Full Fine-Tuning](#2)
  - [ 2.1 - Preprocess the Dialog-Summary Dataset](#2.1)
  - [ 2.2 - Fine-Tune the Model with the Preprocessed Dataset](#2.2)
  - [ 2.3 - Evaluate the Model Qualitatively (Human Evaluation)](#2.3)
  - [ 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#2.4)
- [ 3 - Perform Parameter Efficient Fine-Tuning (PEFT)](#3)
  - [ 3.1 - Setup the PEFT/LoRA model for Fine-Tuning](#3.1)
  - [ 3.2 - Train PEFT Adapter](#3.2)
  - [ 3.3 - Evaluate the Model Qualitatively (Human Evaluation)](#3.3)
  - [ 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#3.4)

<a name='1'></a>
## 1 - Configure o Kernel, Carregue as Dependências Necessárias, Conjunto de Dados e LLM.

:<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies (SOMENTE PARA SAGEMAKER)

In [ ]:
import os

instance_type_expected = 'ml-m5-2xlarge'
instance_type_current = os.environ.get('HOSTNAME')

print(f'Expected instance type: instance-datascience-{instance_type_expected}')
print(f'Currently chosen instance type: {instance_type_current}')

assert instance_type_expected in instance_type_current, f'ERROR. You selected the {instance_type_current} instance type. Please select {instance_type_expected} instead as shown on the screenshot above'
print("Instance type has been chosen correctly.")

Expected instance type: instance-datascience-ml-m5-2xlarge
Currently chosen instance type: instance-datascience-ml-m5-2xlarge
Instance type has been chosen correctly.


In [ ]:
# Instala a versão 2.17.0 da biblioteca datasets
%pip install -U datasets==2.17.0

# Atualiza a versão do pip
%pip install --upgrade pip

# Instalação silenciosa das versões específicas de torch e torchdata
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

# Instalação silenciosa das bibliotecas especificadas
%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

#-- quiet oculta algumas informações que seriam exibidas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importe as bibliotecas necessárias.

In [ ]:
#importando as bibliotecas
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.2'></a>
### 1.2 - Carregando o Dataset e o LLM

Carregando o conjunto de dados [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) da Hugging Face. Ele contém mais de 10.000 diálogos com os resumos manualmente rotulados correspondentes e tópicos.




In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Carregue o modelo pré-treinado [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) e seu tokenizador diretamente da HuggingFace. Observe que você estará utilizando a versão pequena [small version](https://huggingface.co/google/flan-t5-base) do FLAN-T5. Definir `torch_dtype=torch.bfloat16` especifica o tipo de memória a ser usado por este modelo.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

É possível extrair o número de parâmetros do modelo e descobrir quantos deles são treináveis. A seguinte função pode ser usada para fazer isso.

In [ ]:
# Define uma função para imprimir o número de parâmetros treináveis e não treináveis do modelo especificado
def print_number_of_trainable_model_parameters(model):
    # Inicializa contadores para o total de parâmetros treináveis e não treináveis
    trainable_model_params = 0
    all_model_params = 0

    # Itera sobre todos os parâmetros do modelo, somando o número total de parâmetros e contando os parâmetros treináveis
    for _, param in model.named_parameters():
        all_model_params += param.numel()  # Adiciona o número total de parâmetros
        if param.requires_grad:  # Verifica se o parâmetro requer gradiente (ou seja, é treinável)
            trainable_model_params += param.numel()  # Se for treinável, adiciona ao contador de parâmetros treináveis

    # Retorna uma string formatada contendo informações sobre os parâmetros do modelo
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# Chama a função com o modelo original e imprime o resultado
print(print_number_of_trainable_model_parameters(original_model))


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


<a name='1.3'></a>
### 1.3 - Teste do Modelo com Zero Shot Inferencia


Teste o modelo com a inferência de zero shot. Você verá que o modelo tem dificuldade em resumir o diálogo em comparação com o resumo de referência, mas ele consegue extrair algumas informações importantes do texto, o que indica que o modelo pode ser ajustado para a tarefa em questão.

In [ ]:
# Define o índice do exemplo a ser utilizado
index = 200

# Extrai o diálogo e o resumo do conjunto de dados para o índice especificado
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

# Cria o prompt para a geração do resumo, incorporando o diálogo
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokeniza o prompt para que seja compreensível pelo modelo, retornando tensores PyTorch
inputs = tokenizer(prompt, return_tensors='pt')

# Gera a saída (resumo) utilizando o modelo original
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

# Cria uma linha de traços para fins de formatação
dash_line = '-'.join('' for x in range(100))

# Imprime uma linha de traços seguida pelo prompt, resumo humano de referência, e o resumo gerado pelo modelo
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

<a name='2'></a>
## 2 - Performance do Fine-Tuning completo

<a name='2.1'></a>
### 2.1 - Pré-processar o Conjunto de Dados Diálogo-Sumário

Treine a sequência de diálogo-sumário (prompt-resposta) convertendo-a em instruções explícitas para o LLM. Adicione uma instrução ao início do diálogo com `Summarize the following conversation` e ao início do summary com `Summary` conforme segue:

Prompt de Treinamento (diálogo):

```
Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary:
```

Prompt de Treinamento (summary):
```
Both Chris and Antje participated in the conversation.
```


Em seguida, pré-processar o conjunto de dados de prompt-resposta em tokens e extrair seus `input_ids` (1 por token).

In [ ]:
# Define uma função para tokenizar os exemplos do conjunto de dados
def tokenize_function(example):
    # Define o início e o fim do prompt para a geração de resumo
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    # Cria um prompt para cada diálogo no exemplo e adiciona o prompt de resumo
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    # Tokeniza os prompts e os resumos, adicionando 'input_ids' e 'labels' aos exemplos
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# O conjunto de dados na verdade contém 3 divisões diferentes: treino, validação, teste.
# O código da função tokenize_function lida com todos os dados em todas as divisões em lotes.
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove as colunas 'id', 'topic', 'dialogue' e 'summary' dos conjuntos de dados tokenizados
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Para economizar tempo, vamos subamostrar o conjunto de dados:

In [ ]:
# Filtra os exemplos dos conjuntos de dados tokenizados mantendo apenas aqueles cujos índices são múltiplos de 100
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)


Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Verifique as formas de todas as três partes do conjunto de dados:

In [ ]:
# Imprime os formatos dos conjuntos de dados
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

# Imprime os conjuntos de dados tokenizados
print(tokenized_datasets)


Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


A saida do dataset ja esta pronto para o fine-tuning.

<a name='2.2'></a>
### 2.2 - Fine-Tuning do Modelo com o Conjunto de Dados Pré-processado

Agora utilize a classe `Trainer` incorporada da Hugging Face (consulte a documentação [here](https://huggingface.co/docs/transformers/main_classes/trainer)). Passe o conjunto de dados pré-processado com referência ao modelo original. Outros parâmetros de treinamento são encontrados experimentalmente e não há necessidade de entrar em detalhes sobre eles no momento.

In [ ]:
# Define o diretório de saída para salvar os resultados do treinamento, incluindo checkpoints e logs,
# usando um nome baseado na data e hora atual
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

# Define os argumentos de treinamento, incluindo o diretório de saída, a taxa de aprendizado,
# o número de épocas de treinamento, o decaimento de peso, o intervalo de registro de logs e
# o número máximo de passos (batches)
training_args = TrainingArguments(
    output_dir=output_dir,  # Diretório de saída para salvar os resultados do treinamento
    learning_rate=1e-5,     # Taxa de aprendizado
    num_train_epochs=1,     # Número de épocas de treinamento
    weight_decay=0.01,      # Decaimento de peso (regularização L2)
    logging_steps=1,        # Intervalo de registro de logs durante o treinamento
    max_steps=1             # Número máximo de passos (batches) de treinamento
)

# Define o treinador que será responsável por executar o treinamento do modelo
trainer = Trainer(
    model=original_model,                    # Modelo a ser treinado
    args=training_args,                     # Argumentos de treinamento definidos anteriormente
    train_dataset=tokenized_datasets['train'],   # Conjunto de dados de treinamento
    eval_dataset=tokenized_datasets['validation'] # Conjunto de dados de validação
)


In [ ]:
# Inicia o processo de treinamento do modelo utilizando o treinador definido anteriormente
trainer.train()

Step,Training Loss
1,49.250000


TrainOutput(global_step=1, training_loss=49.25, metrics={'train_runtime': 70.1219, 'train_samples_per_second': 0.114, 'train_steps_per_second': 0.014, 'total_flos': 5478058819584.0, 'train_loss': 49.25, 'epoch': 0.06})

Treinar uma versão completamente ajustada do modelo levaria algumas horas em uma GPU. Para economizar tempo, faça o download de um ponto de verificação do modelo totalmente ajustado para usar no restante deste notebook. Este modelo totalmente ajustado também será referido como o **modelo instruído**.

In [ ]:
# Copia recursivamente os arquivos do bucket S3 "dlai-generative-ai/models/flan-dialogue-summary-checkpoint/"
# para o diretório local "./flan-dialogue-summary-checkpoint/"
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/generation_config.json to flan-dialogue-summary-checkpoint/generation_config.json
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/scheduler.pt to flan-dialogue-summary-checkpoint/scheduler.pt
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/rng_state.pth to flan-dialogue-summary-checkpoint/rng_state.pth
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/config.json to flan-dialogue-summary-checkpoint/config.json
download: s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/trainer_state.json to flan-dialogue-su

O tamanho do modelo instruído baixado é de aproximadamente 1 GB.

In [ ]:
# Lista informações detalhadas do arquivo "pytorch_model.bin" no diretório "./flan-dialogue-summary-checkpoint/"
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-r--r-- 1 root root 945M May 15  2023 ./flan-dialogue-summary-checkpoint/pytorch_model.bin


Crie uma instância da classe `AutoModelForSeq2SeqLM` para o modelo instruído:






In [ ]:
# Carrega um modelo de geração de sequências de resumo a partir do diretório local "./flan-dialogue-summary-checkpoint"
# usando os parâmetros pré-treinados encontrados neste diretório e especifica a precisão de ponto flutuante como bfloat16
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)


<a name='2.3'></a>
### 2.3 - Avalie o Modelo Qualitativamente (Avaliação Humana)

Como em muitas aplicações de IA generativa, uma abordagem qualitativa em que você se pergunta "Meu modelo está se comportando como deveria?" geralmente é um bom ponto de partida. No exemplo abaixo (o mesmo com o qual começamos este notebook), você pode ver como o modelo ajustado é capaz de criar um resumo razoável do diálogo em comparação com a incapacidade original de entender o que está sendo pedido ao modelo.

In [ ]:
# Define o índice do exemplo a ser utilizado
index = 200

# Extrai o diálogo e o resumo humano de referência do conjunto de dados para o índice especificado
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

# Cria um prompt para a geração de resumo, incorporando o diálogo
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokeniza o prompt para que seja compreensível pelos modelos, retornando tensores PyTorch
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Gera resumos utilizando o modelo original e o modelo de instrução
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Imprime os resultados: resumo humano de referência, resumo gerado pelo modelo original e resumo gerado pelo modelo de instrução
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: You'd like to upgrade your computer. #Person2: You'd like to upgrade your computer.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# suggests #Person2# upgrading #Person2#'s system, hardware, and CD-ROM drive. #Person2# thinks it's great.


<a name='2.4'></a>
### 2.4 - Avalie o Modelo Quantitativamente (com a Métrica ROUGE)

A [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) ajuda a quantificar a validade das sumarizações produzidas pelos modelos. Ela compara as sumarizações com um resumo "baseline" que geralmente é criado por um humano. Embora não seja perfeita, ela indica o aumento geral na eficácia de sumarização que conseguimos alcançar através do fine-tuning.

In [ ]:
# Carrega o módulo de avaliação para ROUGE (ROUGE-N, ROUGE-L, ROUGE-W) usando a biblioteca evaluate
rouge = evaluate.load('rouge')


Gere as saídas para a amostra do conjunto de dados de teste (apenas 10 diálogos e sumários para economizar tempo) e salve os resultados.

In [ ]:
# Extrai os diálogos e resumos humanos de referência para os primeiros 10 exemplos do conjunto de dados de teste
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Inicializa listas para armazenar os resumos gerados pelos modelos original e de instrução
original_model_summaries = []
instruct_model_summaries = []

# Itera sobre cada diálogo para gerar resumos usando os modelos original e de instrução
for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Gera resumo usando o modelo original
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    # Gera resumo usando o modelo de instrução
    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

# Agrupa os resumos em uma lista de tuplas
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

# Cria um DataFrame pandas com os resumos gerados e os resumos humanos de referência
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df


,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: Thank you for your time.,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,This memo should go out as an intra-office mem...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees who use the Instant Messaging progra...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,#Person1: I'm sorry you're stuck in traffic. #...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1#: I'm finally here. I've got a traffi...,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,The driver of the car is stuck in a traffic jam.,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting married.,Masha and Hero are getting divorced. Kate can'...
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Happy birthday, Brian. #Person2#: H...",Brian's birthday is coming. #Person1# invites ...



Avalie os modelos calculando as métricas ROUGE. Observe a melhoria nos resultados!

In [ ]:
# Calcula as métricas ROUGE (ROUGE-N, ROUGE-L, ROUGE-W) para os resumos gerados pelo modelo original
# usando os resumos humanos de referência correspondentes como referência
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE (ROUGE-N, ROUGE-L, ROUGE-W) para os resumos gerados pelo modelo de instrução
# usando os resumos humanos de referência correspondentes como referência
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Imprime os resultados do modelo original
print('ORIGINAL MODEL:')
print(original_model_results)

# Imprime os resultados do modelo de instrução
print('INSTRUCT MODEL:')
print(instruct_model_results)


ORIGINAL MODEL:
{'rouge1': 0.24223171760013867, 'rouge2': 0.10614243734192583, 'rougeL': 0.21380459196706333, 'rougeLsum': 0.21740921541379205}
INSTRUCT MODEL:
{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}


O arquivo `data/dialogue-summary-training-results.csv` contém uma lista pré-populada de todos os resultados do modelo que você pode usar para avaliar em uma seção maior de dados. Vamos fazer isso para cada um dos modelos:

In [ ]:
# Lê os resultados do arquivo CSV "data/dialogue-summary-training-results.csv" usando pandas
results = pd.read_csv("data/dialogue-summary-training-results.csv")

# Extrai os resumos humanos de referência, os resumos gerados pelo modelo original e os resumos gerados pelo modelo de instrução
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

# Calcula as métricas ROUGE (ROUGE-N, ROUGE-L, ROUGE-W) para os resumos gerados pelo modelo original
# usando os resumos humanos de referência correspondentes como referência
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE (ROUGE-N, ROUGE-L, ROUGE-W) para os resumos gerados pelo modelo de instrução
# usando os resumos humanos de referência correspondentes como referência
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Imprime os resultados do modelo original
print('ORIGINAL MODEL:')
print(original_model_results)

# Imprime os resultados do modelo de instrução
print('INSTRUCT MODEL:')
print(instruct_model_results)


ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}


Os resultados mostram uma melhoria substancial em todas as métricas ROUGE:

In [ ]:
# Imprime o título para indicar que os resultados exibidos são a melhoria percentual absoluta do MODELO DE INSTRUÇÃO sobre o MODELO ORIGINAL
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

# Calcula a melhoria percentual absoluta do MODELO DE INSTRUÇÃO sobre o MODELO ORIGINAL para cada métrica ROUGE
improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    # Imprime a métrica ROUGE e sua melhoria percentual absoluta
    print(f'{key}: {value*100:.2f}%')


Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 18.82%
rouge2: 10.43%
rougeL: 13.70%
rougeLsum: 13.69%


<a name='3'></a>
## 3 -  Fine-Tuning Eficiente de Parâmetros (PEFT).

Agora, vamos realizar o **Fine-Tuning Eficiente de Parâmetros (PEFT)** em vez do "fine-tuning completo" como feito anteriormente. O PEFT é uma forma de fine-tuning instrucional que é muito mais eficiente do que o fine-tuning completo - com resultados de avaliação comparáveis, como você verá em breve.

O PEFT é um termo genérico que inclui a **Adaptação de Baixa Ordem (LoRA)** e a sintonia de prompt (que NÃO É A MESMA que a engenharia de prompt!). Na maioria dos casos, quando alguém menciona PEFT, eles geralmente estão se referindo ao LoRA. LoRA, em um nível muito alto, permite ao usuário ajustar seu modelo usando menos recursos computacionais (em alguns casos, uma única GPU). Após o ajuste fino para uma tarefa específica, caso de uso ou inquilino com LoRA, o resultado é que o LLM original permanece inalterado e um "adaptador LoRA" recém-treinado surge. Este adaptador LoRA é muito, muito menor que o LLM original - da ordem de % de dígitos únicos do tamanho original do LLM (MBs versus GBs).

Dito isso, no momento da inferência, o adaptador LoRA precisa ser reunido e combinado com seu LLM original para atender à solicitação de inferência. O benefício, no entanto, é que muitos adaptadores LoRA podem reutilizar o LLM original, o que reduz os requisitos de memória globais ao atender a várias tarefas e casos de uso.

<a name='3.1'></a>
### 3.1 - Configure o modelo PEFT/LoRA para Fine-Tuning.

Você precisa configurar o modelo PEFT/LoRA para o fine-tuning com uma nova camada/adaptador de parâmetros. Usando o PEFT/LoRA, você está congelando o LLM subjacente e treinando apenas o adaptador. Dê uma olhada na configuração do LoRA abaixo. Observe o hiperparâmetro de rank (`r`), que define o rank/dimensão do adaptador a ser treinado.

In [ ]:
# Importa as bibliotecas necessárias do pacote peft
from peft import LoraConfig, get_peft_model, TaskType

# Define a configuração do modelo LORA (Lightweight Optimized Rank Adaptation)
lora_config = LoraConfig(
    r=32,                       # Rank
    lora_alpha=32,              # Hiperparâmetro alpha do LORA
    target_modules=["q", "v"],  # Módulos-alvo para aplicar a adaptação de rank
    lora_dropout=0.05,          # Taxa de dropout do LORA
    bias="none",                # Tipo de viés (bias)
    task_type=TaskType.SEQ_2_SEQ_LM  # Tipo de tarefa (neste caso, FLAN-T5, uma tarefa de modelo de linguagem seq2seq)
)


Adicione camadas/adaptadores LoRA aos parâmetros originais do LLM a serem treinados.

In [ ]:
# Obtém o modelo adaptado usando o método get_peft_model do pacote peft, fornecendo o modelo original e a configuração do LORA
peft_model = get_peft_model(original_model, lora_config)

# Imprime o número de parâmetros treináveis do modelo adaptado usando a função print_number_of_trainable_model_parameters
print(print_number_of_trainable_model_parameters(peft_model))


trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


<a name='3.2'></a>
### 3.2 - Treine o Adaptador PEFT.

Defina os argumentos de treinamento e crie uma instância do Trainer.

In [ ]:
# Define o diretório de saída para salvar os resultados do treinamento do modelo PEFT
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

# Define os argumentos de treinamento para o modelo PEFT, incluindo o diretório de saída,
# a taxa de aprendizado, o número de épocas de treinamento, o intervalo de registro de logs e
# o número máximo de passos (batches)
peft_training_args = TrainingArguments(
    output_dir=output_dir,          # Diretório de saída para salvar os resultados do treinamento
    auto_find_batch_size=True,      # Encontrar automaticamente o tamanho do lote ideal
    learning_rate=1e-3,             # Taxa de aprendizado (mais alta do que o ajuste fino completo)
    num_train_epochs=1,             # Número de épocas de treinamento
    logging_steps=1,                # Intervalo de registro de logs durante o treinamento
    max_steps=1                     # Número máximo de passos (batches) de treinamento
)

# Define o treinador para o modelo PEFT, que será responsável por executar o treinamento do modelo
peft_trainer = Trainer(
    model=peft_model,               # Modelo PEFT a ser treinado
    args=peft_training_args,       # Argumentos de treinamento definidos anteriormente
    train_dataset=tokenized_datasets["train"],  # Conjunto de dados de treinamento
)


In [ ]:
# Inicia o processo de treinamento do modelo PEFT
peft_trainer.train()

# Define o caminho para salvar o modelo PEFT treinado e o tokenizador
peft_model_path="./peft-dialogue-summary-checkpoint-local"

# Salva o modelo PEFT treinado no diretório especificado
peft_trainer.model.save_pretrained(peft_model_path)

# Salva o tokenizador no mesmo diretório
tokenizer.save_pretrained(peft_model_path)


Step,Training Loss
1,51.250000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

O treinamento foi realizado em um subconjunto de dados. Para carregar um modelo PEFT totalmente treinado, leia um ponto de verificação de um modelo PEFT do S3.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/tokenizer_config.json to peft-dialogue-summary-checkpoint-from-s3/tokenizer_config.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/adapter_config.json to peft-dialogue-summary-checkpoint-from-s3/adapter_config.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/special_tokens_map.json to peft-dialogue-summary-checkpoint-from-s3/special_tokens_map.json
download: s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/adapter_model.bin to peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin
download: s3://dlai-generative-ai/mo

Verifique se o tamanho deste modelo é muito menor do que o do LLM original:

In [ ]:
!ls -al ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-r--r-- 1 root root 14208525 May 15  2023 ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin


Prepare este modelo adicionando um adaptador ao modelo original FLAN-T5. Você está definindo `is_trainable=False` porque o plano é apenas realizar inferência com este modelo PEFT. Se você estivesse preparando o modelo para mais treinamento, você definiria `is_trainable=True`.

In [ ]:
# Importa as classes necessárias do pacote peft
from peft import PeftModel, PeftConfig

# Carrega o modelo base FLAN-T5 e o tokenizador associado
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Carrega o modelo PEFT especificando o modelo base, o diretório de checkpoints, o tipo de tensor e se o modelo é treinável
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)


O número de parâmetros treináveis será `0` devido à configuração `is_trainable=False`:

In [ ]:
# Imprime o número de parâmetros treináveis do modelo PEFT
print(print_number_of_trainable_model_parameters(peft_model))


trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


<a name='3.3'></a>
### 3.3 - Avalie o Modelo Qualitativamente (Avaliação Humana)

Faça inferências para o mesmo exemplo das seções [1.3](#1.3) e [2.3](#2.3), com o modelo original, totalmente ajustado e o modelo PEFT.

In [ ]:
# Define o índice do diálogo a ser resumido
index = 200

# Obtém o diálogo e o resumo humano de referência correspondente do conjunto de dados de teste
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

# Monta o prompt para resumir o diálogo
prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

# Converte o prompt em IDs de tokens usando o tokenizador
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Gera o resumo usando o modelo original
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Gera o resumo usando o modelo de instrução
instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

# Gera o resumo usando o modelo PEFT
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Imprime os resumos gerados e o resumo humano de referência
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Pork1: Have you considered upgrading your system? #Person1: Yes, but I'd like to make some improvements. #Pork1: I'd like to make a painting program. #Person1: I'd like to make a flyer. #Person1: I'd like to make a banner. #Person2: I'd like to make a banner. #Person1: I'd like to make a fly. #Person2: I'd like to make a CD-ROM drive. #Person1: I'd like to make a CD-ROM drive.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# suggests #Person2# upgrading #Person2#'s system, hardware, and CD-ROM drive. #Person2# thinks it's great.
-----------------------------------------------------------

<a name='3.4'></a>
### 3.4 - Avalie o Modelo Quantitativamente (com a Métrica ROUGE)
Realize inferências para a amostra do conjunto de dados de teste (apenas 10 diálogos e sumários para economizar tempo).

In [ ]:
# Obtém os diálogos e os resumos humanos de referência dos primeiros 10 exemplos do conjunto de dados de teste
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Inicializa listas vazias para armazenar os resumos gerados pelos modelos original, de instrução e PEFT
original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

# Itera sobre os diálogos e seus índices no conjunto de dados
for idx, dialogue in enumerate(dialogues):
    # Monta o prompt para resumir o diálogo
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    # Converte o prompt em IDs de tokens usando o tokenizador
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Obtém o resumo humano de referência correspondente ao índice atual
    human_baseline_text_output = human_baseline_summaries[idx]

    # Gera o resumo usando o modelo original
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    # Gera o resumo usando o modelo de instrução
    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    # Gera o resumo usando o modelo PEFT
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    # Adiciona os resumos gerados pelas três abordagens aos respectivos listas
    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

# Combina os resumos humanos de referência e os resumos gerados pelos modelos em uma lista de tuplas
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

# Cria um DataFrame a partir das tuplas, nomeando as colunas
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])

# Retorna o DataFrame contendo os resumos gerados pelos modelos e os resumos humanos de referência
df


,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,This is a memo to all employees.,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: This is an intra-office memo. #Pers...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam in the Carrefour area is a pro...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1: I'm finally here! #Person2: I'm fina...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
5,#Person2# complains to #Person1# about the tra...,People are complaining about the way the car i...,#Person2# got stuck in traffic again. #Person1...,#Person2# got stuck in traffic and #Person1# s...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,The divorce is being finalized.,Masha and Hero are getting divorced. Kate can'...,Kate tells #Person2# Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,Brian's birthday party was a great success.,Brian's birthday is coming. #Person1# invites ...,Brian remembers his birthday and invites #Pers...


In [ ]:
# Carrega o módulo de avaliação ROUGE
rouge = evaluate.load('rouge')

# Calcula as métricas ROUGE para os resumos gerados pelo modelo original
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE para os resumos gerados pelo modelo de instrução
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE para os resumos gerados pelo modelo PEFT
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Imprime os resultados de ROUGE para cada modelo
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)


ORIGINAL MODEL:
{'rouge1': 0.26528933794361415, 'rouge2': 0.11408233383308347, 'rougeL': 0.2496716571243882, 'rougeLsum': 0.2542598261388923}
INSTRUCT MODEL:
{'rouge1': 0.41026607717457186, 'rouge2': 0.17840645241958838, 'rougeL': 0.2977022096267017, 'rougeLsum': 0.2987374187518165}
PEFT MODEL:
{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}


Observe que os resultados do modelo PEFT não são tão ruins, enquanto o processo de treinamento foi muito mais fácil!

Você já calculou o escore ROUGE no conjunto de dados completo, após carregar os resultados do arquivo `data/dialogue-summary-training-results.csv`. Carregue os valores para o modelo PEFT agora e verifique seu desempenho em comparação com outros modelos.

In [ ]:
# Obtém os resumos gerados pelos modelos e os resumos humanos de referência do DataFrame de resultados
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

# Calcula as métricas ROUGE para os resumos gerados pelo modelo original
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE para os resumos gerados pelo modelo de instrução
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Calcula as métricas ROUGE para os resumos gerados pelo modelo PEFT
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

# Imprime os resultados de ROUGE para cada modelo
print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)


ORIGINAL MODEL:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
INSTRUCT MODEL:
{'rouge1': 0.42161291557556113, 'rouge2': 0.18035380596301792, 'rougeL': 0.3384439349963909, 'rougeLsum': 0.33835653595561666}
PEFT MODEL:
{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}


Os resultados mostram menos melhoria em relação ao fine-tuning completo, mas os benefícios do PEFT geralmente superam as métricas de desempenho ligeiramente mais baixas.

Calcule a melhoria do PEFT em relação ao modelo original:

In [ ]:
# Imprime a melhoria percentual absoluta do modelo PEFT em relação ao modelo original
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

# Calcula a diferença entre as métricas ROUGE do modelo PEFT e do modelo original
improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))

# Itera sobre as métricas ROUGE e suas melhorias percentuais absolutas
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')


Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 17.47%
rouge2: 8.73%
rougeL: 12.36%
rougeLsum: 12.34%


Agora, calcule a melhoria do PEFT em relação a um modelo totalmente ajustado:

In [ ]:
# Imprime a melhoria percentual absoluta do modelo PEFT em relação ao modelo de instrução
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

# Calcula a diferença entre as métricas ROUGE do modelo PEFT e do modelo de instrução
improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))

# Itera sobre as métricas ROUGE e suas melhorias percentuais absolutas
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')


Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -1.35%
rouge2: -1.70%
rougeL: -1.34%
rougeLsum: -1.35%


Aqui você vê uma pequena diminuição percentual nas métricas ROUGE em comparação com o fine-tuning completo. No entanto, o treinamento requer muito menos recursos computacionais e de memória (geralmente apenas uma única GPU).